# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import random
from data import Data, STATS_COLUMNS
import model
import elo

torch.manual_seed(20250222)
random.seed(20250222)
np.random.seed(20250222)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
dataset = Data(batch_size=100)

dataset.games.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,...,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000
mean,2015.575896,70.371624,2101.677721,74.190697,2099.323680,61.205668,0.061738,26.180557,57.072441,6.919323,...,19.266279,11.437051,16.819468,10.801152,21.937435,11.218625,15.191485,6.454031,2.849489,18.836014
std,6.073611,36.089915,986.770174,11.403172,989.989621,11.373177,0.287218,4.809783,7.820486,3.168482,...,6.327385,5.238806,6.985267,4.417465,4.706791,3.764948,5.029754,2.984269,2.036512,4.586616
min,2003.000000,0.000000,1101.000000,30.000000,1101.000000,11.000000,0.000000,9.000000,26.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2011.000000,38.000000,1261.000000,66.000000,1254.000000,54.000000,0.000000,23.000000,52.000000,5.000000,...,15.000000,8.000000,12.000000,8.000000,19.000000,9.000000,12.000000,4.000000,1.000000,16.000000
50%,2016.000000,74.000000,1413.000000,74.000000,1407.000000,61.000000,0.000000,26.000000,57.000000,7.000000,...,19.000000,11.000000,16.000000,10.000000,22.000000,11.000000,15.000000,6.000000,3.000000,19.000000
75%,2021.000000,101.000000,3245.000000,81.000000,3246.000000,69.000000,0.000000,29.000000,62.000000,9.000000,...,23.000000,15.000000,21.000000,14.000000,25.000000,14.000000,18.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,3480.000000,149.000000,3480.000000,144.000000,6.000000,58.000000,113.000000,30.000000,...,80.000000,48.000000,65.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The x's will be the indexes of two team IDs and program IDs, the y's will include and indicator of who won at the game stats

## Predicting Stats
The first model we build will predict game statistics. There is more information encapsulated in the box scores than simply wins and losses so we will use this to build up embeddings that are predictive about how the games will play out

In [3]:
stats_train_loader, stats_validation_loader = dataset.train_test_data(output_stats=True, cache="stats", train_size=0.99)

Loading cached data


In [4]:
stats_model = model.StatsModel(program_embedding=512, team_embedding=512,
                               num_programs=len(dataset.programs), num_teams=len(dataset.teams),
                               model_sizes=(512, 256), dropout=0.5).to(device)

In [5]:
model.train(stats_train_loader, stats_validation_loader, stats_model, name="stats_model", learning_rate=0.001, use_cache=True)

Loading from cache


## The Model
Now we use the embeddings developed in the first model to build a model to predict which team will win, which is the thing we are, in the end trying to do.

In [6]:
result_train_loader, result_validation_loader = dataset.train_test_data(output_stats=False, cache="result", train_size=0.99, seed=20250320)

Loading cached data


In [7]:
result_model = model.Model(matchup=stats_model.matchup, model_sizes=(384, 64), dropout=0.5).to(device)
result_model.matchup.freeze()

## Training the model

Train the model

In [8]:
model.train(result_train_loader, result_validation_loader, result_model, name="result_model", learning_rate=0.0001)

Epoch   0: Train Loss=0.24952979, Test Loss=0.24954304
Epoch   1: Train Loss=0.16384998, Test Loss=0.16688902
Epoch   2: Train Loss=0.16364914, Test Loss=0.16687318
Epoch   3: Train Loss=0.16343311, Test Loss=0.16700908
Epoch   4: Train Loss=0.16386590, Test Loss=0.16731908
Epoch   5: Train Loss=0.16347014, Test Loss=0.16696801
Epoch   6: Train Loss=0.16384493, Test Loss=0.16762440
Epoch   7: Train Loss=0.16168635, Test Loss=0.16576605
Epoch   8: Train Loss=0.16357340, Test Loss=0.16767171
Epoch   9: Train Loss=0.16140582, Test Loss=0.16539340
Epoch  10: Train Loss=0.16281094, Test Loss=0.16674005
Epoch  11: Train Loss=0.16151233, Test Loss=0.16537046
Epoch  12: Train Loss=0.16246119, Test Loss=0.16643479
Epoch  13: Train Loss=0.16358856, Test Loss=0.16764157
Epoch  14: Train Loss=0.16210076, Test Loss=0.16608303
Epoch  15: Train Loss=0.16299207, Test Loss=0.16733678
Epoch  16: Train Loss=0.16302327, Test Loss=0.16745245
Best Loss: 0.165370


In [9]:
model.print_results(result_validation_loader, result_model, label="Validation Results")

Validation Results: Accuracy=75.55, Loss=0.165370


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [10]:
tourney_dataset = dataset.gen_dataset(dataset.tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=500, shuffle=True)

In [11]:
model.print_results(tourney_loader, result_model, label="Tournament Results")

Tournament Results: Accuracy=73.77, Loss=0.169583


When it comes to tournament results we get a little worse. The lower result is likely due to teams having increased pairity.

In [12]:
stage1_loader = dataset.tourney_data(after=2021)
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=72.22, Loss=0.175968


Train with early torney data

In [13]:
tourney_df = dataset.tourney[dataset.tourney.Season < 2021]

tourney_train_df, tourney_validation_df = train_test_split(tourney_df, train_size=0.8)
tourney_train_data = dataset.gen_dataset(tourney_train_df)
tourney_validation_data = dataset.gen_dataset(tourney_validation_df)

tourney_train_loader = DataLoader(tourney_train_data, batch_size=500)
tourney_validation_loader = DataLoader(tourney_validation_data, batch_size=500)

In [14]:
result_model.matchup.freeze()

In [15]:
model.train(tourney_train_loader, tourney_validation_loader, result_model, name="tuned_model", learning_rate=0.00001)

Epoch   0: Train Loss=0.16546630, Test Loss=0.17662290
Epoch   1: Train Loss=0.16545785, Test Loss=0.17661168
Epoch   2: Train Loss=0.16543528, Test Loss=0.17659603
Epoch   3: Train Loss=0.16540999, Test Loss=0.17658096
Epoch   4: Train Loss=0.16538620, Test Loss=0.17656759
Epoch   5: Train Loss=0.16536161, Test Loss=0.17655691
Epoch   6: Train Loss=0.16533993, Test Loss=0.17654604
Epoch   7: Train Loss=0.16531776, Test Loss=0.17654030
Epoch   8: Train Loss=0.16530167, Test Loss=0.17653581
Epoch   9: Train Loss=0.16528862, Test Loss=0.17653391
Epoch  10: Train Loss=0.16527576, Test Loss=0.17653423
Epoch  11: Train Loss=0.16526148, Test Loss=0.17653588
Epoch  12: Train Loss=0.16525004, Test Loss=0.17653739
Epoch  13: Train Loss=0.16524082, Test Loss=0.17653682
Epoch  14: Train Loss=0.16523103, Test Loss=0.17653600
Best Loss: 0.176534


In [16]:
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=72.41, Loss=0.175946


### Performance by year


In [17]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(year=season)
    model.print_results(loader, result_model, label=f"{season} Tournament")

2003 Tournament: Accuracy=77.34, Loss=0.166988
2004 Tournament: Accuracy=71.09, Loss=0.183239
2005 Tournament: Accuracy=74.22, Loss=0.177916
2006 Tournament: Accuracy=67.97, Loss=0.202736
2007 Tournament: Accuracy=75.78, Loss=0.175399
2008 Tournament: Accuracy=78.91, Loss=0.167004
2009 Tournament: Accuracy=73.44, Loss=0.154179
2010 Tournament: Accuracy=70.87, Loss=0.171937
2011 Tournament: Accuracy=73.08, Loss=0.171794
2012 Tournament: Accuracy=75.77, Loss=0.158762
2013 Tournament: Accuracy=71.15, Loss=0.176945
2014 Tournament: Accuracy=74.23, Loss=0.163316
2015 Tournament: Accuracy=80.00, Loss=0.141707
2016 Tournament: Accuracy=74.62, Loss=0.171323
2017 Tournament: Accuracy=78.85, Loss=0.154087
2018 Tournament: Accuracy=68.46, Loss=0.189347
2019 Tournament: Accuracy=76.15, Loss=0.149333
2021 Tournament: Accuracy=71.32, Loss=0.178091
2022 Tournament: Accuracy=74.25, Loss=0.172679
2023 Tournament: Accuracy=69.78, Loss=0.188652
2024 Tournament: Accuracy=74.25, Loss=0.164444


Breaking out by league

In [18]:
for season in dataset.tourney.Season.unique():
    for league in dataset.tourney[dataset.tourney.Season == season].League.unique():
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, result_model, label=f"{season} {league} Tournament")

2003 M Tournament: Accuracy=77.34, Loss=0.166988
2004 M Tournament: Accuracy=71.09, Loss=0.183239
2005 M Tournament: Accuracy=74.22, Loss=0.177916
2006 M Tournament: Accuracy=67.97, Loss=0.202736
2007 M Tournament: Accuracy=75.78, Loss=0.175399
2008 M Tournament: Accuracy=78.91, Loss=0.167004
2009 M Tournament: Accuracy=73.44, Loss=0.154179
2010 M Tournament: Accuracy=67.19, Loss=0.201895
2010 W Tournament: Accuracy=74.60, Loss=0.141504
2011 M Tournament: Accuracy=66.42, Loss=0.220170
2011 W Tournament: Accuracy=80.16, Loss=0.120346
2012 M Tournament: Accuracy=69.40, Loss=0.187889
2012 W Tournament: Accuracy=82.54, Loss=0.127786
2013 M Tournament: Accuracy=65.67, Loss=0.203179
2013 W Tournament: Accuracy=76.98, Loss=0.149045
2014 M Tournament: Accuracy=66.42, Loss=0.202810
2014 W Tournament: Accuracy=82.54, Loss=0.121315
2015 M Tournament: Accuracy=77.61, Loss=0.160232
2015 W Tournament: Accuracy=82.54, Loss=0.122005
2016 M Tournament: Accuracy=73.88, Loss=0.187399
2016 W Tournament: A

## Inspect the model
First what are the sizes of the smallest input and output weights

In [19]:
print(f"Program embedding min: {result_model.matchup.program_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"Team embedding min: {result_model.matchup.team_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {result_model.fc3.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Program embedding min: 2.373395
Team embedding min: 3.467458
FC min: 0.005858


Calculate the average gradient for each input feature

In [20]:
result_model.matchup.unfreeze()

In [21]:
program_weights, team_weights, stats_weights = model.feature_eval(result_model, tourney_loader)

In [22]:
program_weights.abs().sum().item(), team_weights.abs().sum().item()

(0.0002655542630236596, 0.00024688150733709335)

In [23]:
print(f"Year:\t{stats_weights[0]:>4f}")
print(f"Game:\t{stats_weights[1]:>4f}")
print(f"League:\t{stats_weights[2]:>4f}")

Year:	0.000000
Game:	-0.000128
League:	0.001935


## Save the model

In [24]:
torch.save(result_model.state_dict(), 'model.pth')

## Moderated model

Moderate a model by pushing it towards 0.5

In [25]:
moderated = model.ModeratedModel(result_model, 0.75)
model.print_results(stage1_loader, moderated, label="Moderated Model")

Moderated Model: Accuracy=72.41, Loss=0.180513


In [26]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(season)
    model.print_results(loader, moderated, label=f"{season} Tournament")

2003 Tournament: Accuracy=77.34, Loss=0.175176
2004 Tournament: Accuracy=71.09, Loss=0.187803
2005 Tournament: Accuracy=74.22, Loss=0.182195
2006 Tournament: Accuracy=67.97, Loss=0.203827
2007 Tournament: Accuracy=75.78, Loss=0.183262
2008 Tournament: Accuracy=78.91, Loss=0.176497
2009 Tournament: Accuracy=73.44, Loss=0.166375
2010 Tournament: Accuracy=70.87, Loss=0.177327
2011 Tournament: Accuracy=73.08, Loss=0.176194
2012 Tournament: Accuracy=75.77, Loss=0.167994
2013 Tournament: Accuracy=71.15, Loss=0.181418
2014 Tournament: Accuracy=74.23, Loss=0.170366
2015 Tournament: Accuracy=80.00, Loss=0.154094
2016 Tournament: Accuracy=74.62, Loss=0.176490
2017 Tournament: Accuracy=78.85, Loss=0.163055
2018 Tournament: Accuracy=68.46, Loss=0.188568
2019 Tournament: Accuracy=76.15, Loss=0.159569
2021 Tournament: Accuracy=71.32, Loss=0.181009
2022 Tournament: Accuracy=74.25, Loss=0.178961
2023 Tournament: Accuracy=69.78, Loss=0.190811
2024 Tournament: Accuracy=74.25, Loss=0.171288


It underperforms even in years where there were upsets.

## Dig into 2023 results

In [27]:
loader = dataset.tourney_data(2023)

x, y = loader.dataset.tensors

preds = result_model(x.to(device))

In [28]:
t_2023 = pd.DataFrame({'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                       'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                       'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                       'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                       'actual': y.reshape(-1),
                       'predicted': preds.cpu().detach().numpy().reshape(-1)}).iloc[:67]

In [29]:
t_2023[t_2023.predicted < 0.5].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted
23,F Dickinson,Purdue,1192,1345,1.0,0.018710
15,Princeton,Arizona,1343,1112,1.0,0.096016
8,Furman,Virginia,1202,1438,1.0,0.178230
53,FL Atlantic,Tennessee,1194,1397,1.0,0.244084
58,San Diego St,Alabama,1361,1104,1.0,0.274868
39,Princeton,Missouri,1343,1281,1.0,0.296200
57,Miami FL,Houston,1274,1222,1.0,0.302939
50,Michigan St,Marquette,1277,1266,1.0,0.383669
14,Penn St,Texas A&M,1336,1401,1.0,0.401231
37,Arkansas,Kansas,1116,1242,1.0,0.410089


The biggest thing in this season were the huge upsets in the first round. Purdue was a number one seed and lost which I only gave a 1% chance to happen. Arizona and Virginia were number 2 seeds and lost which I gave 13% and 17% chances of happening respectively.

In [30]:
t_2023['Upset'] = [dataset.upset(2023, winner, loser) for (winner, loser) in zip(t_2023['winner'], t_2023['loser'])]

In [31]:
t_2023[t_2023.Upset].predicted.mean()

np.float64(0.3699206977142088)

On average the upsets had a 37% chance of happening

In [32]:
t_2023[t_2023.Upset & (t_2023.predicted >= 0.5)].sort_values('predicted', ascending=False)

,winner_name,loser_name,winner,loser,actual,predicted,Upset
45,Creighton,Baylor,1166,1124,1.0,0.577829,True
60,Connecticut,Gonzaga,1163,1211,1.0,0.566729,True
49,Miami FL,Indiana,1274,1231,1.0,0.507195,True


I correctly predicted 4 upsets, though all were closely ranked

In [33]:
t_2023[~t_2023.Upset & (t_2023.predicted < 0.5)].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted,Upset
63,San Diego St,Creighton,1361,1166,1.0,0.419716,False
12,Missouri,Utah St,1281,1429,1.0,0.465533,False
0,Pittsburgh,Mississippi St,1338,1280,1.0,0.466472,False
2,Arizona St,Nevada,1113,1305,1.0,0.468647,False
5,Arkansas,Illinois,1116,1228,1.0,0.476974,False
59,Texas,Xavier,1400,1462,1.0,0.481520,False
55,Kansas St,Michigan St,1243,1277,1.0,0.484676,False


I also incorrectly predicted 5 upsets

Looking at all the tourneys

In [34]:
x, y = tourney_loader.dataset.tensors
preds = result_model(x.to(device))
tourney_df = pd.DataFrame({'season': x[:,4].tolist(),
                           'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                           'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                           'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                           'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                           'actual': y.reshape([-1]),
                           'predicted': np.array(preds.tolist()).reshape([-1])})
tourney_df = tourney_df[tourney_df.actual == 1.0]
tourney_df['Upset'] = [dataset.upset(season, winner, loser) for (winner, loser, season)
                       in zip(tourney_df['winner'], tourney_df['loser'], tourney_df['season'])]

In [35]:
len(tourney_df[tourney_df.Upset & (tourney_df.predicted >= 0.5) & (tourney_df.season > 2020)].sort_values('predicted', ascending=False))

29

In [36]:
len(tourney_df[~tourney_df.Upset & (tourney_df.predicted < 0.5) & (tourney_df.season > 2020)].sort_values('predicted'))

39

Overall I predicted 25 upsets correctly, and 36 incorrectly

## Predicting by seeds
What if I predict just using the seeds?

In [37]:
odds = dataset.odds_by_seed_diff(before=2021)

In [38]:
dataset.tourney_df(after=2021).SeedDiff.map(lambda x: odds[x]**2).mean()

np.float64(0.18527460145235355)

This results in a test Brier score of about 0.185.

## Using Elo
I can also compute Elo rankings

In [39]:
elo_model = elo.EloModel(dataset, k=50)
model.print_results(stage1_loader, elo_model, label="Elo Model")

Elo Model: Accuracy=73.45, Loss=0.178886


It is similar to our model

## Hybrid Model
Building a model using the neural net and seeds

In [40]:
seed_model = model.SeedModel(dataset)
model.print_results(stage1_loader, seed_model, label="Seed Model")
model.print_results(stage1_loader, elo_model, label="Elo Model")
model.print_results(stage1_loader, result_model, label="Base NN")

Seed Model: Accuracy=72.03, Loss=0.184022
Elo Model: Accuracy=73.45, Loss=0.178886
Base NN: Accuracy=72.41, Loss=0.175946


In [41]:
hybrid = model.HybridModel([result_model, seed_model, elo_model], [0.7, 0.2, 0.1])

model.print_results(stage1_loader, hybrid, label="Hybrid Model")

Hybrid Model: Accuracy=73.35, Loss=0.174767


They hybrid model outperforms all individual models

In [42]:
for season in range(2021, 2025):
    for league in ['M', 'W']:
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, hybrid, label=f"{season} {league} Tournament")

2021 M Tournament: Accuracy=68.94, Loss=0.208920
2021 W Tournament: Accuracy=76.98, Loss=0.148420
2022 M Tournament: Accuracy=70.15, Loss=0.203967
2022 W Tournament: Accuracy=77.61, Loss=0.142581
2023 M Tournament: Accuracy=68.66, Loss=0.204670
2023 W Tournament: Accuracy=76.12, Loss=0.166905
2024 M Tournament: Accuracy=65.67, Loss=0.192711
2024 W Tournament: Accuracy=82.84, Loss=0.128901


## Generating the submission file
### Phase 2

Write the results

In [43]:
model.gen_submission(hybrid, dataset, fname="hybrid.csv")

In [44]:
model.gen_submission(result_model, dataset, fname="base.csv")

## Generate a bracket

With the hybrid model

In [45]:
season, league = 2025, 'M'

In [46]:
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.988545
R1W2    1104         Alabama  0.901317
R1W3    1458       Wisconsin  0.922428
R1W4    1112         Arizona  0.865854
R1W5    1332          Oregon  0.761146
R1W6    1140             BYU  0.563347
R1W7    1388    St Mary's CA  0.638845
R1W8    1280  Mississippi St  0.501853
R1X1    1222         Houston  0.980536
R1X2    1397       Tennessee  0.945282
R1X3    1246        Kentucky  0.859264
R1X4    1345          Purdue  0.827910
R1X5    1155         Clemson  0.807394
R1X6    1228        Illinois  0.654205
R1X7    1417            UCLA  0.580206
R1X8    1211         Gonzaga  0.612678
R1Y1    1120          Auburn  0.991398
R1Y2    1277     Michigan St  0.945788
R1Y3    1235         Iowa St  0.853873
R1Y4    1401       Texas A&M  0.763280
R1Y5    1276        Michigan  0.626640
R1Y6    1314  North Carolina  0.522028
R1Y7    1266       Marquette  0.590047
R1Y8    1257      Louisvi

In [47]:
model.tiebreaker(bracket, stats_model, dataset, season, league)

140.15078814263353

And with the base model

In [48]:
base_bracket = model.gen_bracket(dataset, 2025, 'M', result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(base_bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.989738
R1W2    1104         Alabama  0.880963
R1W3    1458       Wisconsin  0.920310
R1W4    1112         Arizona  0.872895
R1W5    1332          Oregon  0.757838
R1W6    1140             BYU  0.523774
R1W7    1388    St Mary's CA  0.621937
R1W8    1124          Baylor  0.503354
R1X1    1222         Houston  0.977154
R1X2    1397       Tennessee  0.940266
R1X3    1246        Kentucky  0.832365
R1X4    1345          Purdue  0.814198
R1X5    1155         Clemson  0.813531
R1X6    1228        Illinois  0.666892
R1X7    1417            UCLA  0.563648
R1X8    1211         Gonzaga  0.620728
R1Y1    1120          Auburn  0.992736
R1Y2    1277     Michigan St  0.941991
R1Y3    1235         Iowa St  0.820584
R1Y4    1401       Texas A&M  0.733190
R1Y5    1276        Michigan  0.570024
R1Y6    1314  North Carolina  0.569749
R1Y7    1266       Marquette  0.572964
R1Y8    1257      Louisvi

In [49]:
model.tiebreaker(base_bracket, stats_model, dataset, season, league)

140.15078814263353